In [1]:
# fetch dataset name list
import os
hhm_path = 'data/hhblits_example/'
pdb_path = 'data/pdb_example/'
hhm_path_files = os.listdir(hhm_path)  
name_list = []
for fi in hhm_path_files: 
    hhm_name = fi.split('.')[0]
    name_list.append(hhm_name)
print(len(name_list))

1742


In [2]:
# one-hot dict for proteins
import numpy as np
eyes = np.eye(20)
protein_dict = {'C':eyes[0], 'D':eyes[1], 'S':eyes[2], 'Q':eyes[3], 'K':eyes[4],
        'I':eyes[5], 'P':eyes[6], 'T':eyes[7], 'F':eyes[8], 'N':eyes[9],
        'G':eyes[10], 'H':eyes[11], 'L':eyes[12], 'R':eyes[13], 'W':eyes[14],
        'A':eyes[15], 'V':eyes[16], 'E':eyes[17], 'Y':eyes[18], 'M':eyes[19]}
#print(protein_dict)

In [20]:
# generate sliding window dataset, size = 61 (30+1+30)
window_length = 61
shhm_path = 'data/shhm_data/5A_simple_average_shhm/'
all_sample_x = []
all_sample_y = []
t = int((window_length - 1) / 2)
with open("data/dataset_alphafold.txt") as file:
    line = file.readline()
    while line:
        label_list = []
        if(line[0] == '>'):
            uniprot_id = line[1:].strip()
            seq = file.readline().strip()
            label = file.readline().strip()
            seq_len = len(seq)
            feature_matrix = np.zeros([seq_len + 2 * t, 52], float) # 20+30+1+1,onehot+spacehhblits+noseq+mask
            shhm_matrix = np.loadtxt(shhm_path + uniprot_id + ".shhm")
            # check padding
            for i in range(feature_matrix.shape[0]):
                if(i < t):
                    feature_matrix[i][-2] = 1 #noseq
                elif(i >= seq_len + t):
                    feature_matrix[i][-2] = 1 #noseq
                else:
                    feature_matrix[i,0:20] = protein_dict[seq[i - t]]
                    feature_matrix[i,20:50] = shhm_matrix[i - t,:]
                    feature_matrix[i,-1] = 1 # mask
        #print(feature_matrix.shape) # seq_len + 2 * t, 52
        # sliding window
        top = 0
        buttom = window_length
        while(buttom <= feature_matrix.shape[0]):
            all_sample_x.append(feature_matrix[top:buttom])            
            top += 1
            buttom += 1
        for i in range(seq_len):
            all_sample_y.append(int(label[i]))
        #print(np.array(sample_list).shape)
        #print(np.array(label_list).shape)
        line = file.readline()        
all_sample_x = np.array(all_sample_x)
all_sample_y = np.array(all_sample_y)
print(all_sample_x.shape)
print(all_sample_y.shape)
np.save("data/dataset_5A_simple_average_shhm_slidingwindow_x.npy", all_sample_x)
np.save("data/dataset_5A_simple_average_shhm_slidingwindow_y.npy", all_sample_y)

(1007941, 61, 52)
(1007941,)


In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [4]:
# set input layers
input_feature = tf.keras.layers.Input(shape=[61, 51], name = 'input_feature')
input_mask = tf.keras.layers.Input(shape=[61,], name = 'input_mask')

In [5]:
# 1 test MLP
# build model
hidden_1 = tf.keras.layers.Dense(512, activation='relu')(input_feature)
hidden_2 = tf.keras.layers.Dense(256, activation='relu')(hidden_1)
drop1 = tf.keras.layers.Dropout(0.3)(hidden_2)
hidden_3 = tf.keras.layers.BatchNormalization()(drop1)
hidden_4 = tf.keras.layers.Dense(128, activation='relu')(hidden_3)
output = tf.keras.layers.Dense(2, activation='softmax', name = 'output_MLP')(hidden_4)
model_MLP = tf.keras.models.Model(inputs=input_feature, outputs=output)
model_MLP.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_feature (InputLayer)  [(None, 61, 51)]          0         
                                                                 
 dense (Dense)               (None, 61, 512)           26624     
                                                                 
 dense_1 (Dense)             (None, 61, 256)           131328    
                                                                 
 dropout (Dropout)           (None, 61, 256)           0         
                                                                 
 batch_normalization (BatchN  (None, 61, 256)          1024      
 ormalization)                                                   
                                                                 
 dense_2 (Dense)             (None, 61, 128)           32896     
                                                             

In [7]:
# 2 test CNN
# build model
hidden_1 = tf.keras.layers.Conv1D(32, 5, kernel_initializer='he_uniform')(input_feature)
hidden_1 = tf.keras.layers.BatchNormalization()(hidden_1)
hidden_1 = tf.keras.layers.Activation('relu')(hidden_1)
hidden_1 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=None)(hidden_1)
hidden_2 = tf.keras.layers.Conv1D(32, 7, kernel_initializer='he_uniform')(hidden_1)
hidden_2 = tf.keras.layers.BatchNormalization()(hidden_2)
hidden_2 = tf.keras.layers.Activation('relu')(hidden_2)
hidden_2 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=None)(hidden_2)
hidden_3 = tf.keras.layers.Conv1D(32, 7, kernel_initializer='he_uniform')(input_feature)
hidden_3 = tf.keras.layers.BatchNormalization()(hidden_3)
hidden_3 = tf.keras.layers.Activation('relu')(hidden_3)
hidden_3 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=None)(hidden_3)
hidden_3 = tf.keras.layers.Flatten()(hidden_3)
output = tf.keras.layers.Dense(128, activation='relu')(hidden_3)
output = tf.keras.layers.Dense(2, activation='softmax', name = 'output_CNN')(output)
model_CNN = tf.keras.models.Model(inputs=input_feature, outputs=output)
model_CNN.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_feature (InputLayer)  [(None, 61, 51)]          0         
                                                                 
 conv1d_5 (Conv1D)           (None, 55, 32)            11456     
                                                                 
 batch_normalization_6 (Batc  (None, 55, 32)           128       
 hNormalization)                                                 
                                                                 
 activation_5 (Activation)   (None, 55, 32)            0         
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 27, 32)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 864)               0   

In [8]:
# 3 test RNN
# build model
units = 32
rnn = tf.keras.layers.SimpleRNN(units,return_sequences=True)(input_feature)
rnn = tf.keras.layers.SimpleRNN(units,return_sequences=True)(rnn)
rnn = tf.keras.layers.BatchNormalization()(rnn)
rnn = tf.keras.layers.Flatten()(rnn)
#print('bet_cnn.get_shape()', rnn.get_shape())
rnn = tf.keras.layers.Dense(128, activation='relu')(rnn)
output = tf.keras.layers.Dense(2, activation='softmax', name = 'output_RNN')(rnn)
model_RNN = tf.keras.models.Model(inputs=input_feature, outputs=output)
model_RNN.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_feature (InputLayer)  [(None, 61, 51)]          0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 61, 32)            2688      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 61, 32)            2080      
                                                                 
 batch_normalization_7 (Batc  (None, 61, 32)           128       
 hNormalization)                                                 
                                                                 
 flatten_2 (Flatten)         (None, 1952)              0         
                                                                 
 dense_5 (Dense)             (None, 128)               249984    
                                                           

In [9]:
# 4 test LSTM
# build model
units = 32
lstm = tf.keras.layers.LSTM(units, return_sequences=True)(input_feature)
lstm = tf.keras.layers.LSTM(units, return_sequences=True)(lstm)
lstm = tf.keras.layers.BatchNormalization()(lstm)
lstm = tf.keras.layers.Flatten()(lstm)
#print('lstm.get_shape()', lstm.get_shape())
lstm = tf.keras.layers.Dense(128, activation='relu')(lstm)
output = tf.keras.layers.Dense(2, activation='softmax', name = 'output_LSTM')(lstm)
model_LSTM = tf.keras.models.Model(inputs=input_feature, outputs=output)
model_LSTM.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_feature (InputLayer)  [(None, 61, 51)]          0         
                                                                 
 lstm (LSTM)                 (None, 61, 32)            10752     
                                                                 
 lstm_1 (LSTM)               (None, 61, 32)            8320      
                                                                 
 batch_normalization_8 (Batc  (None, 61, 32)           128       
 hNormalization)                                                 
                                                                 
 flatten_3 (Flatten)         (None, 1952)              0         
                                                                 
 dense_6 (Dense)             (None, 128)               249984    
                                                           

In [10]:
# 5 test BiLSTM
units = 32
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, return_sequences=True))(input_feature)
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, return_sequences=True))(lstm)
lstm = tf.keras.layers.BatchNormalization()(lstm)
lstm = tf.keras.layers.Flatten()(lstm)
#print('lstm.get_shape()', lstm.get_shape())
lstm = tf.keras.layers.Dense(128, activation='relu')(lstm)
output = tf.keras.layers.Dense(2, activation='softmax', name = 'output_BiLSTM')(lstm)
model_BiLSTM = tf.keras.models.Model(inputs=input_feature, outputs=output)
model_BiLSTM.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_feature (InputLayer)  [(None, 61, 51)]          0         
                                                                 
 bidirectional (Bidirectiona  (None, 61, 64)           21504     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 61, 64)           24832     
 nal)                                                            
                                                                 
 batch_normalization_9 (Batc  (None, 61, 64)           256       
 hNormalization)                                                 
                                                                 
 flatten_4 (Flatten)         (None, 3904)              0         
                                                           

In [11]:
# 6 test CNN+BiLSTM
units = 32
hidden_1 = tf.keras.layers.Conv1D(32, 5, kernel_initializer='he_uniform')(input_feature)
hidden_1 = tf.keras.layers.BatchNormalization()(hidden_1)
hidden_1 = tf.keras.layers.Activation('relu')(hidden_1)
hidden_1 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=None)(hidden_1)
hidden_2 = tf.keras.layers.Conv1D(32, 7, kernel_initializer='he_uniform')(hidden_1)
hidden_2 = tf.keras.layers.BatchNormalization()(hidden_2)
hidden_2 = tf.keras.layers.Activation('relu')(hidden_2)
hidden_2 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=None)(hidden_2)
hidden_3 = tf.keras.layers.Conv1D(32, 7, kernel_initializer='he_uniform')(input_feature)
hidden_3 = tf.keras.layers.BatchNormalization()(hidden_3)
hidden_3 = tf.keras.layers.Activation('relu')(hidden_3)
hidden_3 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=None)(hidden_3)
#print('hidden_3.get_shape()', hidden_3.get_shape())
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, return_sequences=True))(hidden_3)
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, return_sequences=True))(lstm)
lstm = tf.keras.layers.BatchNormalization()(lstm)
lstm = tf.keras.layers.Flatten()(lstm)
#print('lstm.get_shape()', lstm.get_shape())
lstm = tf.keras.layers.Dense(128, activation='relu')(lstm)
output = tf.keras.layers.Dense(2, activation='softmax', name = 'output_CNN_BiLSTM')(lstm)
model_CNN_BiLSTM = tf.keras.models.Model(inputs=input_feature, outputs=output)
model_CNN_BiLSTM.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_feature (InputLayer)  [(None, 61, 51)]          0         
                                                                 
 conv1d_8 (Conv1D)           (None, 55, 32)            11456     
                                                                 
 batch_normalization_12 (Bat  (None, 55, 32)           128       
 chNormalization)                                                
                                                                 
 activation_8 (Activation)   (None, 55, 32)            0         
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 27, 32)           0         
 1D)                                                             
                                                                 
 bidirectional_2 (Bidirectio  (None, 27, 64)           1664

In [13]:
# 7 test Transformer
from utils.Transformer import MultiHeadSelfAttention
from utils.Transformer import TransformerBlock
from utils.Transformer import TokenAndPositionEmbedding

def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    # add extra dimensions to add the padding
    # to the attention logits.
    return  seq[:, tf.newaxis, tf.newaxis, :]# (batch_size, 1, 1, seq_len)

maxlen = 1024
vocab_size = 5
embed_dim = 64
num_heads = 4
ff_dim = 64
pos_embed_dim = 64
seq_embed_dim = 13
num_heads = 4

embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim, pos_embed_dim, seq_embed_dim)
trans_block_1 = TransformerBlock(embed_dim, num_heads, ff_dim)
trans_block_2 = TransformerBlock(embed_dim, num_heads, ff_dim)

mask = create_padding_mask(input_mask)
embedding = embedding_layer([input_mask, input_feature])
embedding = trans_block_1(embedding, mask)
embedding = trans_block_2(embedding, mask)
#print('embedding.get_shape()', embedding.get_shape())

transformer = tf.keras.layers.Flatten()(embedding)
transformer = tf.keras.layers.Dense(128, activation='relu')(transformer)
output = tf.keras.layers.Dense(2, activation='softmax', name = 'output_Transformer')(transformer)
model_Transformer = tf.keras.models.Model(inputs=[input_feature,input_mask], outputs=output)
model_Transformer.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_mask (InputLayer)        [(None, 61)]         0           []                               
                                                                                                  
 tf.math.equal_1 (TFOpLambda)   (None, 61)           0           ['input_mask[0][0]']             
                                                                                                  
 input_feature (InputLayer)     [(None, 61, 51)]     0           []                               
                                                                                                  
 tf.cast_1 (TFOpLambda)         (None, 61)           0           ['tf.math.equal_1[0][0]']        
                                                                                            